In [1]:
# from svglib.svglib import svg2rlg
# from reportlab.graphics import renderPM
# from cairosvg import svg2png
from datetime import datetime
from bs4 import BeautifulSoup as BS
from fastapi import FastAPI as FP
from fastapi import HTTPException, Depends, Security
from fastapi.security.api_key import APIKeyQuery, APIKeyCookie, APIKeyHeader, APIKey
from starlette.status import HTTP_403_FORBIDDEN
from starlette.responses import RedirectResponse, JSONResponse
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname
from pydantic import BaseModel
import uvicorn
import boto3
import random
import requests
import string
import os

In [2]:
now = datetime.now()

In [3]:
app = FP()

In [4]:
load_dotenv(find_dotenv())

True

In [5]:
API_KEY = os.environ.get("token")
API_KEY_NAME = "access_token"
api_key_query = APIKeyQuery(name=API_KEY_NAME, auto_error=False)
api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=False)

In [6]:
async def get_api_key(
    api_key_query: str = Security(api_key_query),
    api_key_header: str = Security(api_key_header),
):
    if api_key_query == API_KEY:
        return api_key_query
    elif api_key_header == API_KEY:
        return api_key_header
    else:
        raise HTTPException(
            status_code=HTTP_403_FORBIDDEN, detail="Could not validate credentials"
        )

In [7]:
donnees = {
    "img": [
        "test1",
        "test2",
        "test3"
    ]
}

In [86]:
def detect_labels_local_file(photo, link):

    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
    print('Detected labels in ' + photo)
    donnees[link] = []
    for label in response['Labels']:
        print (label['Name'] + ' : ' + str(label['Confidence']))
        donnees[link].append(label['Name'] + ' : ' + str(label['Confidence']))
        

    return len(response['Labels'])

In [106]:
def analyze(web):
    link = web
    page = requests.get("https://" + link)
    content = BS(page.content, 'html.parser')
    img = content.find_all('img', alt=False)
    for image in img:
        if link in image['src']:
            image = image['src']
            response = requests.get(image)
        else:
            image = link + image['src']
            response = requests.get("https://" + image)
        if image[-3:] == "png" or image[-3:] == "jpg":
            file = open(random.choice(string.ascii_letters) + now.strftime("%H%M%S") + ".jpg", "wb")
            file.write(response.content)
            photo=file.name
            detect_labels_local_file(photo, image)
        # elif image[-3:] == "svg":
        #     SVGcontent = BS(requests.get(image).content, 'html.parser')
        #     SVG = SVGcontent
        #     with open(random.choice(string.ascii_letters) + now.strftime("%H%M%S") + ".svg", "w") as file:
        #         file.write(str(SVG))
        #         print(file.name[:len(file.name) - 4])

In [10]:
@app.get("/token_test")
async def token_validation(api_key: APIKey = Depends(get_api_key)):
    response = "The bot confirm you had a valid Token"
    return {"response": response}

In [11]:
@app.get("/images")
async def get_images(web:str, api_key: APIKey = Depends(get_api_key)):
    analyze(web)
    return {"response": "Valid",
            "data" : donnees,
            "Website" : web
            }

In [ ]:
if __name__ == "__main__":
    uvicorn.run("Recognize:app",
                host="152.228.175.112"
                # ssl-keyfile=os.path.join('.','cert','key.pem')
                # ssl-certfile=os.path.join('.','cert','cert.pem')
                )